In [1]:
import numpy as np
import sklearn
print(np.__version__)
print(sklearn.__version__)


1.26.4
1.8.0


In [2]:
import numpy as np
import pandas as pd
import re
import string

In [3]:
##now let's losd the model that we saved
import pickle
with open('../static/model/model.pickle','rb') as f:
    model = pickle.load(f)

/opt/anaconda3/envs/mlclean/lib/python3.11/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.7.2 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
#remove stopwords(am/is/are/...)
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

stopwords = set(ENGLISH_STOP_WORDS)

def remove_stopwords(text):
    words = text.split()
    cleaned = [w for w in words if w.lower() not in stopwords]
    return " ".join(cleaned)

#remove punctuations
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text



In [6]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [7]:
#stemming(base word[creating->create])

from nltk.stem import PorterStemmer

# Create a PorterStemmer object
ps = PorterStemmer()

In [8]:
def preprosessing(text):
    data=pd.DataFrame([text],columns=['tweet'])
    #convert uppercase to lowercase
    data['tweet']= data['tweet'].apply(lambda x:" ".join(x.lower() for x in x.split()))
    #remove links
    data['tweet']= data['tweet'].apply(lambda x:" ".join(re.sub(r'(https?://\S+|www\.\S+)',' ',x,flags=re.MULTILINE) for x in x.split()))
    #remove punctuations
    data['tweet'] = data['tweet'].apply(remove_punctuations)
    #remove numbers
    data['tweet'] = data['tweet'].str.replace(r'\d+', '', regex=True)
    #remove stopwords(am/is/are/...)
    data['tweet'] = data['tweet'].apply(remove_stopwords)
    #stemming(base word[creating->create])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(word) for word in x.split()))
    return data['tweet']


In [4]:
txt='great product . i like it'

In [9]:
pre_processed_text = preprosessing(txt)

In [10]:
txt ##before

'great product . i like it'

In [11]:
pre_processed_text ##after

0    great product like
Name: tweet, dtype: object

In [12]:
vocab = pd.read_csv('../static/model/vocabulary.txt',header=None)  ##take the vocabulary
tokens = vocab[0].tolist()

In [13]:
tokens

['test',
 'android',
 'app',
 'beauti',
 'cute',
 'health',
 'iger',
 'iphoneonli',
 'iphonesia',
 'iphon',
 'final',
 'case',
 'thank',
 'yay',
 'soni',
 'xperia',
 's',
 'love',
 'talk',
 'relax',
 'smartphon',
 'wifi',
 'connect',
 'm',
 'know',
 'way',
 'home',
 'amaz',
 'servic',
 'appl',
 'won',
 't',
 'question',
 'pay',
 'stupid',
 'support',
 'softwar',
 'updat',
 'fuck',
 'phone',
 'big',
 'time',
 'happi',
 'instap',
 'instadaili',
 'xperiaz',
 'new',
 'type',
 'c',
 'charger',
 'cabl',
 'uk',
 '…',
 'amazon',
 'year',
 'newyear',
 'start',
 'technolog',
 'samsunggalaxi',
 'iphonex',
 'pic',
 'twitter',
 'com',
 'shop',
 'listen',
 'music',
 'likeforlik',
 'photo',
 'fun',
 'selfi',
 'water',
 'camera',
 'picoftheday',
 'sun',
 'instagood',
 'boy',
 'outdoor',
 'hey',
 'make',
 'ipod',
 'dont',
 'color',
 'inch',
 'crash',
 'doe',
 'need',
 'realli',
 'drop',
 'ball',
 'design',
 'give',
 'just',
 'crazi',
 'purchas',
 'lol',
 'work',
 'hard',
 'play',
 'ipad',
 'batteri',
 

In [24]:
##now need to vectorize(make as numbers)
import numpy as np

def vectorizer(ds, vocabulary):
    
    vectorized_lst = [] #final vectorize dictionary came here
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary)) #check whether the word is there and mark as 1 and 0
    
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
        
        vectorized_lst.append(sentence_lst) #appending to the vectorized_lst
    
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    return vectorized_lst_new

In [30]:
def get_prediction(vectorized_txt):
    model.predict(vectorized_txt)
    if prediction == 1:
        return 'Negative'
    else:
        return 'Positive'

In [31]:
#dataset->preprosessed test
#for vocabulary->token
vectorized_txt = vectorizer(pre_processed_text,tokens)

In [32]:
vectorized_txt

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [33]:
#predicting
model.predict(vectorized_txt)

array([1])

In [38]:
test='very bad product, I hate it.'
pre_processed_text = preprosessing(test)
vectorized_txt = vectorizer(pre_processed_text,tokens)
prediction =get_prediction(vectorized_txt) 
prediction

'Positive'

In [39]:
test='good product.I love it.'
pre_processed_text = preprosessing(test)
vectorized_txt = vectorizer(pre_processed_text,tokens)
prediction =get_prediction(vectorized_txt) 
prediction

'Positive'